In [23]:
import docx2txt

def extract_docx(path):
    return docx2txt.process(path)

In [24]:
import os
import fitz  # PyMuPDF
from docx import Document
import re

# ---------- PDF Extraction ----------
def extract_pdf(path):
    text = ""
    doc = fitz.open(path)
    for page in doc:
        text += page.get_text("text") + "\n"
    doc.close()
    return text

# ---------- Word Extraction ----------
def extract_docx(path):
    doc = Document(path)
    return "\n".join([para.text for para in doc.paragraphs])

# ---------- Cleaning Function ----------
def clean_text(text):
    text = re.sub(r'-\n', '', text)   # fix broken words
    text = re.sub(r'\s+', ' ', text)  # normalize spaces
    return text.strip()

# ---------- Process Folder ----------
folder_path = "documents_folder"
documents = []

for file in os.listdir(folder_path):
    path = os.path.join(folder_path, file)

    if file.endswith(".pdf"):
        text = extract_pdf(path)

    elif file.endswith(".docx"):
        text = extract_docx(path)

    else:
        continue

    text = clean_text(text)

    documents.append({
        "filename": file,
        "text": text
    })

print(f"Processed {len(documents)} documents.")

Processed 5 documents.


In [73]:
import os
import fitz  # PyMuPDF for PDFs
from docx import Document
import re

# ---------- PDF Extraction ----------
def extract_pdf(path):
    text = ""
    doc = fitz.open(path)
    for page in doc:
        text += page.get_text("text") + "\n"
    doc.close()
    return text

# ---------- Word Extraction ----------
def extract_docx(path):
    doc = Document(path)
    return "\n".join([para.text for para in doc.paragraphs])

# ---------- Cleaning ----------
def clean_text(text):
    text = re.sub(r'-\n', '', text)    # fix broken words
    text = re.sub(r'\s+', ' ', text)   # normalize spaces
    return text.strip()

# ---------- Section Chunking ----------

def chunk_tribunal_robust(text):
    """
    Splits tribunal text into main sections using keywords.
    Falls back to full text if nothing matches.
    """
    text = re.sub(r'\r\n', '\n', text)
    text = re.sub(r'\s+', ' ', text)

    # Keywords commonly used in rulings
    keywords = [
        r'\b[A-Z]\.\s+[A-Z][A-Za-z0-9 ,\-&()]+',  # A. Background
        'Background', 'Dispute Background',
        'Issues', 'Issues for Determination',
        'Analysis', 'Determination',
        'Orders', 'Conclusion'
    ]

    # Build regex pattern
    pattern = '|'.join(keywords)
    
    matches = list(re.finditer(pattern, text, flags=re.IGNORECASE))
    if not matches:
        # No headings found → treat as one section
        return [{"section": "Full Text", "text": text}]
    
    chunks = []
    for i, match in enumerate(matches):
        start = match.start()
        heading = match.group(0).strip()
        end = matches[i+1].start() if i+1 < len(matches) else len(text)
        section_text = text[start:end].strip()
        chunks.append({"section": heading, "text": section_text})

    return chunks
# ---------- Process Folder ----------
folder_path = "documents_folder"  # replace with your folder path

for file in os.listdir(folder_path):
    path = os.path.join(folder_path, file)

    if not (file.endswith(".pdf") or file.endswith(".docx")):
        continue

    # Extract text
    if file.endswith(".pdf"):
        text = extract_pdf(path)
    else:
        text = extract_docx(path)

    # Clean text
    text = clean_text(text)

    # Chunk by section
    sections = chunk_by_section(text)

    # Print preview
    print(f"\n=== {file} ===")
    print(f"Total sections: {len(sections)}")
    for sec in sections:
        print(f"\n--- {sec['section']} ---")
        print(sec['text'][:300] + "...")  # first 300 chars of each section


=== Esmail v Cine Investment Limited  2 others (Tribunal Case E101of2025) 2025KELAT194(KLR) (8October2025) (Ruling).pdf ===
Total sections: 1

--- R. NABIL M ---
. ORINA PhD CHAIRPERSON Before: Mr. Oichoe for the Claimant Ms. Makana for the 3rd Respondent Lucy – Court Assistant https://new.kenyalaw.org/akn/ke/judgment/kelat/2025/194/eng@2025-10-08 2...

=== Kleen Homes Security Services Ltd v Pet Care Services Limited (Tribunal Case E550E360of2025 (Consolidated)) 2025KEBPRT423(KLR) (9October2025) (Ruling).pdf ===
Total sections: 7

--- A. Background 1 ---
. The Respondent/Landlord led a Notice of Preliminary Objection dated 18th August 2025 seeking to strike out the consolidated references in BPRT E360 of 2025 and BPRT E550 of 2025, contending that the tenancies between the parties are not controlled tenancies within the meaning of Section 2 of the ...

--- B. Issues for Determination 4 ---
. The issues for determination are as follows: i. Whether the Preliminary Objection meets the 

In [74]:
total_sections_all_files = 0

for doc in documents:
    text = doc["text"]
    sections = chunk_tribunal_robust(text)  # split into sections
    num_sections = len(sections)
    total_sections_all_files += num_sections
    print(f"{doc['filename']} → {num_sections} sections")  # optional per-file count

print(f"\nTotal sections across all files: {total_sections_all_files}")

Esmail v Cine Investment Limited  2 others (Tribunal Case E101of2025) 2025KELAT194(KLR) (8October2025) (Ruling).pdf → 7 sections
Kleen Homes Security Services Ltd v Pet Care Services Limited (Tribunal Case E550E360of2025 (Consolidated)) 2025KEBPRT423(KLR) (9October2025) (Ruling).pdf → 14 sections
Ledilas  6 others v National Land Commission Legal Advice Centre ta Kituo Cha Sheria (Amicus Curiae) (Tribunal Case E010of2025) 2026KELAT5(KLR) (9February2026) (Judgment).docx → 29 sections
Munyingi v Thuo (Tribunal Case E105of2024) 2025KEBPRT425(KLR) (9October2025) (Judgment).pdf → 23 sections
T Gas Limited v Baluch  3 others (Tribunal Case E017of2025) 2025KEBPRT424(KLR) (8October2025) (Ruling).pdf → 17 sections

Total sections across all files: 90


In [75]:
for doc in documents:
    text = doc["text"]
    sections = chunk_tribunal_robust(text)  # split text into sections

    print(f"\n=== {doc['filename']} ===")
    print(f"Total sections: {len(sections)}")

    # Print only section names / headings
    for sec in sections:
        print(f"- {sec['section']}")


=== Esmail v Cine Investment Limited  2 others (Tribunal Case E101of2025) 2025KELAT194(KLR) (8October2025) (Ruling).pdf ===
Total sections: 7
- a. The instant suit is similar to another suit which was struck out by this tribunal for lack of jurisdiction being Esmail v Abdeel Enterprises Limited & 3 others
- determination
- b. The present suit similarly invites the Tribunal to venture into matters beyond its jurisdiction
- c. Even though the Claimant purports to
- background
- orders
- M. ORINA PhD CHAIRPERSON Before

=== Kleen Homes Security Services Ltd v Pet Care Services Limited (Tribunal Case E550E360of2025 (Consolidated)) 2025KEBPRT423(KLR) (9October2025) (Ruling).pdf ===
Total sections: 14
- A. Background 1
- B. Issues for Determination 4
- issues
- determination
- i. Whether the Preliminary Objection meets the established legal threshold
- C. Analysis and Determination i
- A. stated
- issues
- t. The provision states that the Tribunal may
- D. Final Orders 23
- a. The Prelimina

In [72]:
# Remove duplicates in the documents list
seen = set()
unique_documents = []

for doc in documents:
    if doc["filename"] not in seen:
        unique_documents.append(doc)
        seen.add(doc["filename"])

documents = unique_documents  # now documents has only unique files

In [ ]:
#documents.append({
    #"filename": file,
   # "text": text
#})
print(documents)

[{'filename': 'Esmail v Cine Investment Limited  2 others (Tribunal Case E101of2025) 2025KELAT194(KLR) (8October2025) (Ruling).pdf', 'text': 'Esmail v Cine Investment Limited & 2 others (Tribunal Case E101 of 2025) [2025] KELAT 194 (KLR) (8 October 2025) (Ruling) Neutral citation: [2025] KELAT 194 (KLR) REPUBLIC OF KENYA IN THE LAND ACQUISITION TRIBUNAL TRIBUNAL CASE E101 OF 2025 NM ORINA, CHAIR OCTOBER 8, 2025 BETWEEN FATMABAI SHERALI ESMAIL ............................................................... CLAIMANT AND CINE INVESTMENT LIMITED .................................................. 1ST RESPONDENT NATIONAL LAND COMMISSION ............................................ 2ND RESPONDENT BUSINESS REGISTRATION SERVICE ...................................... 3RD RESPONDENT RULING 1. This matter comes up for directions today in respect of the Claimant’s Statement of Claim and Application dated 12th September 2025. In the application \ue000led under Certi\ue000cate of Urgency, the Claiman

In [ ]:
#print(documents[:2])
#print(type(documents))

[{'filename': 'Esmail v Cine Investment Limited  2 others (Tribunal Case E101of2025) 2025KELAT194(KLR) (8October2025) (Ruling).pdf', 'text': 'Esmail v Cine Investment Limited & 2 others (Tribunal Case E101 of 2025) [2025] KELAT 194 (KLR) (8 October 2025) (Ruling) Neutral citation: [2025] KELAT 194 (KLR) REPUBLIC OF KENYA IN THE LAND ACQUISITION TRIBUNAL TRIBUNAL CASE E101 OF 2025 NM ORINA, CHAIR OCTOBER 8, 2025 BETWEEN FATMABAI SHERALI ESMAIL ............................................................... CLAIMANT AND CINE INVESTMENT LIMITED .................................................. 1ST RESPONDENT NATIONAL LAND COMMISSION ............................................ 2ND RESPONDENT BUSINESS REGISTRATION SERVICE ...................................... 3RD RESPONDENT RULING 1. This matter comes up for directions today in respect of the Claimant’s Statement of Claim and Application dated 12th September 2025. In the application \ue000led under Certi\ue000cate of Urgency, the Claiman

In [26]:
line = "Kigathi v Kenya National Highways Authority & 2 others (Tribunal Case E071 of 2025) [2025] KELAT 193 (KLR) (29 September 2025) (Judgment)"

metadata = extract_from_citation(line)
print(metadata)

{'case_name': 'Kigathi v Kenya National Highways Authority & 2 others', 'tribunal_case': 'E071 OF 2025', 'neutral_citation': '[2025] KELAT 193', 'decision_date': '29 September 2025'}


In [92]:
import re
import os

def extract_from_filename(filename):
    name = os.path.splitext(filename)[0]

    # Normalize spacing (insert spaces before capital letters if missing)
    clean = re.sub(r'(\d)([A-Z])', r'\1 \2', name)
    clean = re.sub(r'([a-z])([A-Z])', r'\1 \2', clean)
    clean = re.sub(r'\s+', ' ', clean)

    # Case name (before Tribunal Case)
    case_match = re.search(r'^(.*?)\s+\(Tribunal Case', clean)
    case_name = case_match.group(1).strip() if case_match else None

    # Tribunal case
    tribunal_match = re.search(r'E\d+\w*\s*of\s*\d{4}', clean, re.IGNORECASE)
    tribunal_case = tribunal_match.group(0).upper() if tribunal_match else None

    # Neutral citation
    citation_match = re.search(r'\d{4}\s*[A-Z]+\s*\d+', clean)
    neutral_citation = citation_match.group(0) if citation_match else None

    # Date
    date_match = re.search(r'\d{1,2}\s*[A-Za-z]+\s*\d{4}', clean)
    decision_date = date_match.group(0) if date_match else None

    # Extract parties from case name
    petitioner = None
    respondents = []

    if case_name and " v " in case_name:
        parts = case_name.split(" v ")
        petitioner = parts[0].strip()
        respondents = [parts[1].strip()]

    return {
        "case_name": case_name,
        "tribunal_case": tribunal_case,
        "neutral_citation": neutral_citation,
        "decision_date": decision_date,
        "petitioner": petitioner,
        "respondents": respondents
    }

In [96]:
for doc in documents:
    doc['metadata'] = extract_from_filename(doc["filename"])

    print("\n======================")
    print("File:", doc["filename"])
    print(doc['metadata'])


File: Esmail v Cine Investment Limited  2 others (Tribunal Case E101of2025) 2025KELAT194(KLR) (8October2025) (Ruling).pdf
{'case_name': 'Esmail v Cine Investment Limited 2 others', 'tribunal_case': 'E101OF2025', 'neutral_citation': '2025 KELAT194', 'decision_date': '01of2025', 'petitioner': 'Esmail', 'respondents': ['Cine Investment Limited 2 others']}

File: Kleen Homes Security Services Ltd v Pet Care Services Limited (Tribunal Case E550E360of2025 (Consolidated)) 2025KEBPRT423(KLR) (9October2025) (Ruling).pdf
{'case_name': 'Kleen Homes Security Services Ltd v Pet Care Services Limited', 'tribunal_case': 'E360OF2025', 'neutral_citation': '2025 KEBPRT423', 'decision_date': '60of2025', 'petitioner': 'Kleen Homes Security Services Ltd', 'respondents': ['Pet Care Services Limited']}

File: Ledilas  6 others v National Land Commission Legal Advice Centre ta Kituo Cha Sheria (Amicus Curiae) (Tribunal Case E010of2025) 2026KELAT5(KLR) (9February2026) (Judgment).docx
{'case_name': 'Ledilas 6 

In [97]:
for doc in documents:
    print(doc["filename"])
    print(doc.get("metadata", "No metadata attached"))
    print("---")

Esmail v Cine Investment Limited  2 others (Tribunal Case E101of2025) 2025KELAT194(KLR) (8October2025) (Ruling).pdf
{'case_name': 'Esmail v Cine Investment Limited 2 others', 'tribunal_case': 'E101OF2025', 'neutral_citation': '2025 KELAT194', 'decision_date': '01of2025', 'petitioner': 'Esmail', 'respondents': ['Cine Investment Limited 2 others']}
---
Kleen Homes Security Services Ltd v Pet Care Services Limited (Tribunal Case E550E360of2025 (Consolidated)) 2025KEBPRT423(KLR) (9October2025) (Ruling).pdf
{'case_name': 'Kleen Homes Security Services Ltd v Pet Care Services Limited', 'tribunal_case': 'E360OF2025', 'neutral_citation': '2025 KEBPRT423', 'decision_date': '60of2025', 'petitioner': 'Kleen Homes Security Services Ltd', 'respondents': ['Pet Care Services Limited']}
---
Ledilas  6 others v National Land Commission Legal Advice Centre ta Kituo Cha Sheria (Amicus Curiae) (Tribunal Case E010of2025) 2026KELAT5(KLR) (9February2026) (Judgment).docx
{'case_name': 'Ledilas 6 others v Nati

In [88]:
metadata = doc.get("metadata", {})
case_name = metadata.get("case_name", "Unknown Case")

In [98]:
all_sections = []

for doc in documents:
    text = doc["text"]
    metadata = doc.get("metadata", {})  # get the metadata you just stored

    sections = chunk_tribunal_robust(text)  # or your preferred chunking function

    for sec in sections:
        all_sections.append({
            **metadata,        # attach all metadata fields
            "section": sec["section"],
            "text": sec["text"]
        })

In [99]:
print(f"Total sections across all files: {len(all_sections)}\n")

# Preview first few sections
for sec in sections:
    all_sections.append({
        "case_name": case_name,
        "tribunal_case": metadata.get("tribunal_case", ""),
        "neutral_citation": metadata.get("neutral_citation", ""),
        "decision_date": metadata.get("decision_date", ""),
        "petitioner": metadata.get("petitioner", ""),
        "respondents": metadata.get("respondents", ""),
        "section": sec["section"],
        "text": sec["text"]
    })

Total sections across all files: 90



In [100]:
# Preview first 3 sections
for sec in all_sections[:3]:
    print(f"Case Name: {sec['case_name']}")
    print(f"Tribunal Case: {sec['tribunal_case']}")
    print(f"Decision Date: {sec['decision_date']}")
    print(f"Section: {sec['section']}")
    print(f"Text (first 100 chars): {sec['text'][:100]}...\n")

Case Name: Esmail v Cine Investment Limited 2 others
Tribunal Case: E101OF2025
Decision Date: 01of2025
Section: a. The instant suit is similar to another suit which was struck out by this tribunal for lack of jurisdiction being Esmail v Abdeel Enterprises Limited & 3 others
Text (first 100 chars): a. The instant suit is similar to another suit which was struck out by this tribunal for lack of jur...

Case Name: Esmail v Cine Investment Limited 2 others
Tribunal Case: E101OF2025
Decision Date: 01of2025
Section: determination
Text (first 100 chars): determination that the dispute in the case involved a question of the legal status of the then 3rd R...

Case Name: Esmail v Cine Investment Limited 2 others
Tribunal Case: E101OF2025
Decision Date: 01of2025
Section: b. The present suit similarly invites the Tribunal to venture into matters beyond its jurisdiction
Text (first 100 chars): b. The present suit similarly invites the Tribunal to venture into matters beyond its jurisdiction. ...



In [103]:
for doc in documents:
    print(doc.get("filename"))
    print(doc.get("metadata", "No metadata attached"))
    print("---")

Esmail v Cine Investment Limited  2 others (Tribunal Case E101of2025) 2025KELAT194(KLR) (8October2025) (Ruling).pdf
{'case_name': 'Esmail v Cine Investment Limited 2 others', 'tribunal_case': 'E101OF2025', 'neutral_citation': '2025 KELAT194', 'decision_date': '01of2025', 'petitioner': 'Esmail', 'respondents': ['Cine Investment Limited 2 others']}
---
Kleen Homes Security Services Ltd v Pet Care Services Limited (Tribunal Case E550E360of2025 (Consolidated)) 2025KEBPRT423(KLR) (9October2025) (Ruling).pdf
{'case_name': 'Kleen Homes Security Services Ltd v Pet Care Services Limited', 'tribunal_case': 'E360OF2025', 'neutral_citation': '2025 KEBPRT423', 'decision_date': '60of2025', 'petitioner': 'Kleen Homes Security Services Ltd', 'respondents': ['Pet Care Services Limited']}
---
Ledilas  6 others v National Land Commission Legal Advice Centre ta Kituo Cha Sheria (Amicus Curiae) (Tribunal Case E010of2025) 2026KELAT5(KLR) (9February2026) (Judgment).docx
{'case_name': 'Ledilas 6 others v Nati

In [102]:
# store all sections in a single dataset (all_sections) with metadata attached.
all_sections = []

for doc in documents:
    # doc["metadata"] has case_name, tribunal_case, etc.
    metadata = doc["metadata"]

    # chunk the document into sections
    sections = chunk_tribunal_robust(doc["text"])

    # attach metadata to each section
    for sec in sections:
        all_sections.append({
            **metadata,      # all metadata fields
            "section": sec["section"],
            "text": sec["text"]
        })

In [104]:
# ---------- Verify total sections and sample ----------
print(f"Total sections across all files: {len(all_sections)}")
for s in all_sections[:5]:  # preview first 5 sections
    print(f"\nCase: {s['case_name']} | Section: {s['section']}")
    print(s['text'][:200] + "...")

Total sections across all files: 90

Case: Esmail v Cine Investment Limited 2 others | Section: a. The instant suit is similar to another suit which was struck out by this tribunal for lack of jurisdiction being Esmail v Abdeel Enterprises Limited & 3 others
a. The instant suit is similar to another suit which was struck out by this tribunal for lack of jurisdiction being Esmail v Abdeel Enterprises Limited & 3 others [2025] KELAT 173 (KLR), https://new.k...

Case: Esmail v Cine Investment Limited 2 others | Section: determination
determination that the dispute in the case involved a question of the legal status of the then 3rd Respondent who is now the 1st Respondent....

Case: Esmail v Cine Investment Limited 2 others | Section: b. The present suit similarly invites the Tribunal to venture into matters beyond its jurisdiction
b. The present suit similarly invites the Tribunal to venture into matters beyond its jurisdiction. The Claimant purports that the 1st Respondent is engaged in 

In [29]:
print(text[:1000])

T Gas Limited v Baluch & 3 others (Tribunal Case E017 of 2025) [2025] KEBPRT 424 (KLR) (8 October 2025) (Ruling) Neutral citation: [2025] KEBPRT 424 (KLR) REPUBLIC OF KENYA IN THE BUSINESS PREMISES RENT TRIBUNAL TRIBUNAL CASE E017 OF 2025 J OSODO, CHAIR & GAKUHI CHEGE, MEMBER OCTOBER 8, 2025 BETWEEN T GAS LIMITED ...................................................................................... APPLICANT AND OLE SHERO BALUCH .................................................................. 1ST RESPONDENT PETER MWANGI KIINIKI ......................................................... 2ND RESPONDENT MUSTAFA HUSSEIN BALUCH ................................................... 3RD RESPONDENT MORAN AUCTIONEERS ............................................................ 4TH RESPONDENT RULING A. Dispute Background 1. The Tenant/Applicant, led a Reference dated 29th May 2025 under Section 12(4) of the Landlord and Tenant (Shops, Hotels and Catering Establishments) Act, Cap 301, together wi